In [ ]:
!pip install openai-whisper language-tool-python spacy textstat scikit-learn pandas tqdm
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import whisper
import language_tool_python
import spacy
import textstat
import os
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [ ]:
#I am using openai Whisper ASR Model -using the 'large' model for better accuracy
asr_model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [01:11<00:00, 43.0MiB/s]


In [ ]:
#Java for language_tool_python, which depends on Java runtime
!sudo apt-get update
!sudo apt-get install openjdk-17-jdk -y
!java -version

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,659 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,911 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ub

In [ ]:
!pip install language_tool_python

In [ ]:
#Initializing grammar checker
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')
text = "He go to school yesterday."
matches = tool.check(text)
print(len(matches), "issues found.")

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpd8n42q5s.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://internal1.languagetool.org/snapshots/LanguageTool-latest-snapshot.zip to /root/.cache/language_tool_python.


1 issues found.


In [ ]:
#Loading English model in spacy for NLP processing
tool = language_tool_python.LanguageTool('en-US')
nlp = spacy.load("en_core_web_sm")

In [ ]:
# feature extraction from transcript
def extract_grammar_features(text):
    matches = tool.check(text)
    doc = nlp(text)
    num_sentences = len(list(doc.sents))
    num_words = len([token for token in doc if token.is_alpha])
    avg_sentence_length = num_words / num_sentences if num_sentences else 0
    num_errors = len(matches)
    flesch = textstat.flesch_reading_ease(text)

    return {
        "num_errors": num_errors,
        "num_sentences": num_sentences,
        "num_words": num_words,
        "avg_sentence_length": avg_sentence_length,
        "flesch_readability": flesch
    }


In [ ]:
import torch
print(torch.cuda.is_available()) #Checking if CUDA is available for GPU processing

False


In [ ]:
AUDIO_DIR = "audios_train"
LABELS_CSV = "train.csv"
df = pd.read_csv(LABELS_CSV)
features = []
df.head()

,filename,label
0,audio_1261.wav,1.0
1,audio_942.wav,1.5
2,audio_1110.wav,1.5
3,audio_1024.wav,1.5
4,audio_538.wav,2.0


In [ ]:
#This Transcribes each audio file and extracting grammar features
for i, row in tqdm(df.iterrows(), total=len(df)):
    file_path = os.path.join(AUDIO_DIR, row['filename'])
    score = row['label']

    try:
        # Step 1:Transcribe audio using Whisper
        result = asr_model.transcribe(file_path)
        text = result['text']
        # Step 2:Extract features from transcript
        feats = extract_grammar_features(text)
        feats['label'] = score
        feats['filename'] = row['filename']
        features.append(feats)
    except Exception as e:
        print(f"Error processing {row['filename']}: {e}")

100%|██████████| 444/444 [3:05:23<00:00, 25.05s/it]


In [ ]:
feature_df = pd.DataFrame(features)
feature_df

,num_errors,num_sentences,num_words,avg_sentence_length,flesch_readability,label,filename
0,2,3,12,4.000000,92.80,1.0,audio_1261.wav
1,2,2,41,20.500000,76.05,1.5,audio_942.wav
2,3,4,53,13.250000,36.12,1.5,audio_1110.wav
3,12,10,71,7.100000,80.38,1.5,audio_1024.wav
4,14,13,127,9.769231,77.64,2.0,audio_538.wav
...,...,...,...,...,...,...,...
439,6,17,184,10.823529,76.72,5.0,audio_494.wav
440,0,7,78,11.142857,81.02,5.0,audio_363.wav
441,1,6,107,17.833333,78.79,5.0,audio_481.wav
442,69,5,95,19.000000,56.22,5.0,audio_989.wav


In [ ]:
feature_df.to_csv("feature_file.csv", index=False)

In [ ]:
try:
    if feature_df.empty:
        feature_df = pd.read_csv("feature_file.csv")
except NameError:
    feature_df = pd.read_csv("feature_file.csv")
feature_df.head()

,num_errors,num_sentences,num_words,avg_sentence_length,flesch_readability,label,filename
0,2,3,12,4.000000,92.80,1.0,audio_1261.wav
1,2,2,41,20.500000,76.05,1.5,audio_942.wav
2,3,4,53,13.250000,36.12,1.5,audio_1110.wav
3,12,10,71,7.100000,80.38,1.5,audio_1024.wav
4,14,13,127,9.769231,77.64,2.0,audio_538.wav


In [ ]:
import numpy as np
X = feature_df.drop(columns=["label", "filename"])
Y = feature_df["label"]

In [ ]:
Y.head()

,label
0,1.0
1,1.5
2,1.5
3,1.5
4,2.0


In [ ]:
X.head()

,num_errors,num_sentences,num_words,avg_sentence_length,flesch_readability
0,2,3,12,4.000000,92.80
1,2,2,41,20.500000,76.05
2,3,4,53,13.250000,36.12
3,12,10,71,7.100000,80.38
4,14,13,127,9.769231,77.64


In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.1, random_state=42)

In [ ]:
#Build a basic neural network model using Keras
keras.backend.clear_session()
model = keras.Sequential([
    keras.layers.Dense(10000, activation='relu'),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=[keras.metrics.RootMeanSquaredError()]
)
model.fit(X_train,Y_train,epochs=5,batch_size=32,verbose=1)

Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 360ms/step - loss: 7.4950 - root_mean_squared_error: 2.6973
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 406ms/step - loss: 2.6575 - root_mean_squared_error: 1.6239
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 201ms/step - loss: 1.3599 - root_mean_squared_error: 1.1655
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - loss: 1.2871 - root_mean_squared_error: 1.1338
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 184ms/step - loss: 1.2010 - root_mean_squared_error: 1.0957


In [ ]:
val_preds = model.predict(X_test).flatten()
rmse = np.sqrt(mean_squared_error(Y_test,val_preds))
print("RMSE:",rmse)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
RMSE: 1.3521711880836538


In [ ]:
import pickle

In [ ]:
with open ("model",'wb') as f:
    pickle.dump(model,f)

In [ ]:
model = pickle.load(open("model",'rb'))

In [ ]:
TEST_DIR = "audios_test"
test_files = os.listdir(TEST_DIR)
predictions = []

In [ ]:
#This include 3 steps
for file in tqdm(test_files):
    file_path= os.path.join(TEST_DIR,file)
    try:
        # Step 1: Transcribe the test audio
        result = asr_model.transcribe(file_path)
        text= result['text']

        # Step 2: Extract grammar features
        feats=extract_grammar_features(text)
        feats_df = pd.DataFrame([feats])

        # Step 3: Scale features and predict score
        feats_scaled= scaler.transform(feats_df)
        pred_score =model.predict(feats_scaled)[0][0]
        predictions.append({"filename": file, "label": pred_score})
    except Exception as e:
        print(f"Error processing {file}:{e}")

  0%|          | 0/195 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step


  1%|          | 1/195 [00:19<1:03:10, 19.54s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


  1%|          | 2/195 [00:31<48:32, 15.09s/it]  

...

 99%|█████████▉| 193/195 [1:13:03<00:32, 16.49s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


 99%|█████████▉| 194/195 [1:13:11<00:14, 14.03s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


100%|██████████| 195/195 [1:15:31<00:00, 23.24s/it]


In [ ]:
for pred in predictions:
    pred["label"] = max(0, min(5, float(pred["label"])))

In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv("grammar_score_predictions.csv", index=False)
pred_df

,filename,label
0,audio_692.wav,3.529287
1,audio_235.wav,3.026045
2,audio_580.wav,3.255058
3,audio_391.wav,3.297761
4,audio_805.wav,3.143466
...,...,...
190,audio_1291.wav,3.282163
191,audio_1195.wav,3.203412
192,audio_89.wav,3.401304
193,audio_604.wav,3.179716


This project shows how I integrated speech recognition, NLP-based feature engineering, and a deep learning regression model to build an end-to-end grammar scoring system.

>I used OpenAI's Whisper model to transcribe audio files into text.

>Then, I extracted language features from the transcriptions using spaCy (for sentence structure) and LanguageTool (for grammar error detection).

>I also included a readability metric using textstat (Flesch reading ease score).

>These extracted features were used to train a regression model using TensorFlow and Keras to predict a grammar quality score for each transcript.

#about the files saved:

>The extracted features from all training samples are saved in a CSV file named feature_file.csv, which allows easy reuse without reprocessing the audio files.

>The trained model is saved as model, which can be reloaded and used directly for prediction.

#Testing:

>After training, the model was used to predict grammar scores on the audio data provided in kaggle, and scores were generated based on transcriptions and extracted features.

>The predictions are saved in a file called grammar_score_predictions.csv, which contains the predicted grammar quality scores for each test audio file.